In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install OPENPYXL
!{sys.executable} -m pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
import os
import glob
from pitch_outcomes_batter import PitchOutcomes
from zone_locations import label_zone_locations
from expected_runs import compute_RE24
from trackman_processing import *
import warnings

In [3]:
#Removing the future deprecation warnings within file
##Most prevalent with 90% Max EV chunk
###If you feel this is important to keep an eye on, comment out this line
warnings.filterwarnings(action = 'ignore', category = FutureWarning)

In [4]:
inputs = load_data(season = 2023, path = '../Work Files/2023 Rest of NCAA Regular Season 2')
#Change path to correct data folder

In [5]:
#Only use for single combined files only
#inputs = pd.read_csv('Updated 2022 Trackman.csv')
#inputs.to_pickle('../Work Files/Updated 2022 Trackman.pkl')
inputs

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2023-02-16,16:02:04.57,1,1,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,"Hazzard, Bryce",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2023-02-16,16:02:22.92,1,2,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,"Hazzard, Bryce",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2023-02-16,16:02:44.91,1,3,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,"Hazzard, Bryce",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
3,4,2023-02-16,16:03:11.74,1,4,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,"Hazzard, Bryce",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
4,5,2023-02-16,16:03:34.49,1,5,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,"Hazzard, Bryce",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831804,365,2023-04-26,21:26:17.65,4,1,"Dougan, Kolby",1.000114e+09,Right,KAN_JAY,"McDonald, Kyte",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
831805,366,2023-04-26,21:26:32.89,4,2,"Dougan, Kolby",1.000114e+09,Right,KAN_JAY,"McDonald, Kyte",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
831806,367,2023-04-26,21:26:49.37,4,3,"Dougan, Kolby",1.000114e+09,Right,KAN_JAY,"McDonald, Kyte",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
831807,368,2023-04-26,21:27:10.14,4,4,"Dougan, Kolby",1.000114e+09,Right,KAN_JAY,"McDonald, Kyte",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


In [6]:
#Correct for all errors in ID
merger = pd.DataFrame(inputs.groupby('BatterId')).reset_index()

mergerTab = merger.rename({'index':'Index', 0:'BatterId', 1:'ColNames'}, axis = 'columns')


preData = pd.merge(inputs, mergerTab, how = 'inner', on = 'BatterId')

In [7]:
# Read in averages and put it in a df
average_df = pd.DataFrame()
average_df = pd.read_excel('22_top100HitterKPIs.xlsx')

top_avg_heart_swing = average_df["HeartSwing%"]
top_avg_IZ_swing = average_df["IZSwing%"]
top_avg_OZ_take = average_df["OZTake"]
top_avg_correct = average_df["Correct%"]
top_avg_maxEV = average_df["MaxEV"]
top_avg_avgEV = average_df["AvgEV"]
top_avg_HH = average_df["HH%"]
top_avg_MR = average_df["MR%"]
top_avg_FBCon = average_df["FBCon%"]
top_avg_HeartCon = average_df["HeartCon%"]
top_avg_IZCon = average_df["IZCon%"]
top_avg_NonFBCon = average_df["NonFBCon%"]
top_avg_contact = average_df["Contact%"]
top_avg_gb = average_df["GB%"]
top_avg_90 = average_df["EV90"]

In [8]:
# Create Game Column
def create_game_column():
    game = preData['Date'] + ' ' + preData['PitcherTeam']
    preData.insert(0, 'Game', game)
    return preData

create_game_column()

,Game,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,...,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,Index,ColNames
0,2023-02-16 WIN_BUL,1,2023-02-16,16:02:04.57,1,1,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,...,High,High,High,NaN,NaN,NaN,NaN,NaN,1133,PitchNo Date Time PAofI...
1,2023-02-16 WIN_BUL,2,2023-02-16,16:02:22.92,1,2,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,...,High,High,High,NaN,NaN,NaN,NaN,NaN,1133,PitchNo Date Time PAofI...
2,2023-02-16 WIN_BUL,3,2023-02-16,16:02:44.91,1,3,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,...,High,High,High,NaN,NaN,NaN,NaN,NaN,1133,PitchNo Date Time PAofI...
3,2023-02-16 WIN_BUL,4,2023-02-16,16:03:11.74,1,4,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,...,High,High,High,NaN,NaN,NaN,NaN,NaN,1133,PitchNo Date Time PAofI...
4,2023-02-16 WIN_BUL,5,2023-02-16,16:03:34.49,1,5,"Brodersen, Sam",1.000116e+09,Right,WIN_BUL,...,High,High,High,NaN,NaN,NaN,NaN,NaN,1133,PitchNo Date Time PAofI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831289,2023-04-26 LOU_CAJ,270,2023-04-26,20:35:54.22,2,3,"Tate, Ben",1.000113e+09,Left,LOU_CAJ,...,High,High,High,NaN,NaN,NaN,NaN,NaN,332,PitchNo Date Time PAof...
831290,2023-04-26 LOU_CAJ,271,2023-04-26,20:36:10.86,2,4,"Tate, Ben",1.000113e+09,Left,LOU_CAJ,...,High,High,High,NaN,NaN,NaN,NaN,NaN,332,PitchNo Date Time PAof...
831291,2023-04-26 LOU_CAJ,272,2023-04-26,20:36:29.94,2,5,"Tate, Ben",1.000113e+09,Left,LOU_CAJ,...,High,High,High,NaN,NaN,NaN,NaN,NaN,332,PitchNo Date Time PAof...
831292,2023-04-26 LOU_CAJ,273,2023-04-26,20:36:45.63,2,6,"Tate, Ben",1.000113e+09,Left,LOU_CAJ,...,High,High,High,NaN,NaN,NaN,NaN,NaN,332,PitchNo Date Time PAof...


In [9]:
preData = preData.drop(['Index','ColNames'], axis = 'columns')
data = preData

In [10]:
#Update as season progresses to filter out low-AB hitters
##The number is pitches seen

removedId = []
def dataFilter():
    for i in preData['BatterId'].unique():
        df = preData[preData['BatterId'] == i]
        if df.shape[0] < 300:
            removedId.append(i)
    data = preData[~preData.BatterId.isin(removedId)]
    return data

data = dataFilter()

In [11]:
# Hard Hit Percentage (HHP)
maxVelo_dict = data.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].to_dict()

def hardHitPerc(df,batterId):
    
    maxVelo = maxVelo_dict[batterId] # get max velo for batter
    
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85
    
    # filter df
    df = df[df['BatterId'] == batterId]
    
    # compute hard hits
    threshHits = df[df['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
    totalHits = data[data['ExitSpeed'].notna()].shape[0] # counts no. of hits
    return (threshHits * 100 / totalHits) # hard hit percentage

len(data['BatterId'].unique())

974

In [12]:
# Find RE24
#re24 = compute_RE24(data)

In [13]:
data['PitchCall'].unique()

array(['StrikeCalled', 'InPlay', 'BallCalled', 'FoulBall',
       'StrikeSwinging', 'BallinDirt', 'HitByPitch', 'BallIntentional',
       'Undefined', 'StriekSwinging', 'WildPitch', 'SwingingStrike',
       'CatchersInterference', 'SwinginStrike'], dtype=object)

In [14]:
# useful outcome labels
outcomes = {'Swing': ['StrikeSwinging','FoulBall','InPlay'],
            'Take': ['BallCalled','BallIntentional','StrikeCalled','HitByPitch'],
            'Contact': ['FoulBall','InPlay'],
            'IZ': ['Heart','Edge'],
            'OZ': ['Chase','Waste']} 

In [15]:
# Create vectors for metric name and metric value

metricName = []
metricValue = []
gameVect = []
hitterVect = []
dashboardVect = []
compareToAverage = []
dateType = []
dateValue = []
teamVect = []
hitterIDVect = []
handVect = []

wobahitterIDVect = []
wobaValue = []

In [16]:
# Create ZoneType Column
data = label_zone_locations(data)

C:\Users\harri\Work Files\zone_locations.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PlateLocHeight'] = data['PlateLocHeight'].round(3)
C:\Users\harri\Work Files\zone_locations.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PlateLocSide'] = data['PlateLocSide'].round(3)
C:\Users\harri\Work Files\zone_locations.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [17]:
## Plate Decisions
# heart swing %
avg_heartSwing = top_avg_heart_swing.mean()
std_heartSwing = top_avg_heart_swing.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]
    handVect.append(df['BatterSide'].iloc[0])
    heartPitches = df[df['ZoneType'] == 'Heart'].shape[0] # total pitches in heart
    metricName.append('HeartSwing%')
    if heartPitches == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        heartSwing = ((df[(df['PitchCall'].isin(outcomes['Swing'])) & (df['ZoneType'] == 'Heart')].shape[0]) / heartPitches) * 100
        metricValue.append(heartSwing)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (heartSwing >= (avg_heartSwing + std_heartSwing * 1.5)):
            compareToAverage.append(3)
        elif (heartSwing <= (avg_heartSwing - std_heartSwing * 1.5)):
            compareToAverage.append(0)
        elif (heartSwing <= (avg_heartSwing - std_heartSwing)):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [18]:
# in-zone swing %
avg_IZSwing = top_avg_IZ_swing.mean()
std_IZSwing = top_avg_IZ_swing.std()
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]
    handVect.append(df['BatterSide'].iloc[0])
    inZone = df[(df['ZoneType'].isin(outcomes['IZ']))].shape[0] # total pitches thrown IN the zone
    metricName.append('InZoneSwing%')                
    if inZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        IZSwing = ((df[(df['PitchCall'].isin(outcomes['Swing'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]) / inZone) * 100
        metricValue.append(IZSwing)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (IZSwing >= avg_IZSwing + std_IZSwing * 1.5):
            compareToAverage.append(3)
        elif (IZSwing <= avg_IZSwing - std_IZSwing * 1.5):
            compareToAverage.append(0)
        elif (IZSwing <= avg_IZSwing - std_IZSwing):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [19]:
# out-of-zone take %
avg_OZTake = top_avg_OZ_take.mean()
std_OZTake = top_avg_OZ_take.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]
    handVect.append(df['BatterSide'].iloc[0])
    outZone = df[(df['ZoneType'].isin(outcomes['OZ']))].shape[0] # total pitches thrown OUT of the zone
    metricName.append('OutZoneTake%')              
    if outZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        OZTake = ((df[(df['PitchCall'].isin(outcomes['Take'])) & (df['ZoneType'].isin(outcomes['OZ']))].shape[0]) / outZone) * 100
        metricValue.append(OZTake)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (OZTake >= avg_OZTake + std_OZTake * 1.5):
            compareToAverage.append(3)
        elif (OZTake <= avg_OZTake - std_OZTake * 1.5):
            compareToAverage.append(0)
        elif (OZTake <= avg_OZTake - std_OZTake):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [20]:
avg_corrPer = top_avg_correct.mean()
std_corrPer = top_avg_correct.std()

# correct % (= iz swings + oz takes / total)

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    correctSwings = df[(df['PitchCall'].isin(outcomes['Swing'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]
    correctTakes = df[(df['PitchCall'].isin(outcomes['Take'])) & (df['ZoneType'].isin(outcomes['OZ']))].shape[0]
    metricName.append('Correct%')              
    corrPer = ((correctSwings + correctTakes) / df.shape[0]) * 100
    metricValue.append(corrPer) # note denominator is all pitches
    if df['HomeTeam'].iloc[0] == "CLE_TIG":
        opponent = df['AwayTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    else:
        opponent = df['HomeTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    dateType.append('Season')
    dateValue.append('2023')
    hitter = df['Batter']
    hitterVect.append(hitter.iloc[0])
    dashboardVect.append('Plate Decisions')
    team = df['BatterTeam']
    teamVect.append(team.iloc[0])
    if (corrPer >= avg_corrPer + std_corrPer * 1.5):
        compareToAverage.append(3)
    elif (corrPer <= avg_corrPer - std_corrPer * 1.5):
        compareToAverage.append(0)
    elif (corrPer <= avg_corrPer - std_corrPer):
        compareToAverage.append(1)
    else:
        compareToAverage.append(2)

In [21]:
avg_maxEV = top_avg_maxEV.mean()
std_maxEV = top_avg_maxEV.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    df = df[df['PitchCall'] == 'InPlay']
    handVect.append(df['BatterSide'].iloc[0])
    metricName.append('MaxEV')
    maxEV = df.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].mean()
    metricValue.append(maxEV)
    if df['HomeTeam'].iloc[0] == "CLE_TIG":
        opponent = df['AwayTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    else:
        opponent = df['HomeTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    dateType.append('Season')
    dateValue.append('2023')
    hitter = df['Batter']
    hitterVect.append(hitter.iloc[0])
    dashboardVect.append('Quality Contact')
    team = df['BatterTeam']
    teamVect.append(team.iloc[0])
    if (maxEV >= avg_maxEV + std_maxEV * 1.5):
        compareToAverage.append(3)
    elif (maxEV <= avg_maxEV - std_maxEV * 1.5):
        compareToAverage.append(0)
    elif (maxEV <= avg_maxEV - std_maxEV):
        compareToAverage.append(1)
    else:
        compareToAverage.append(2)

# avg EV
avg_avgEV = top_avg_avgEV.mean()
std_avgEV = top_avg_avgEV.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    df = df[df['PitchCall'] == 'InPlay']
    handVect.append(df['BatterSide'].iloc[0])
    metricName.append('AvgEV')             
    avgEV = df['ExitSpeed'].mean()
    metricValue.append(avgEV)
    if df['HomeTeam'].iloc[0] == "CLE_TIG":
        opponent = df['AwayTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    else:
        opponent = df['HomeTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    dateType.append('Season')
    dateValue.append('2023')
    hitter = df['Batter']
    hitterVect.append(hitter.iloc[0])
    dashboardVect.append('Quality Contact')
    team = df['BatterTeam']
    teamVect.append(team.iloc[0])
    if (avgEV >= avg_avgEV + std_avgEV * 1.5):
        compareToAverage.append(3)
    elif (avgEV <= avg_avgEV - std_avgEV * 1.5):
        compareToAverage.append(0)
    elif (avgEV <= avg_avgEV - std_avgEV):
        compareToAverage.append(1)
    else:
        compareToAverage.append(2)
        

In [22]:
# 90% percentile Max EV
avg_90 = top_avg_90.mean()
std_90 = top_avg_90.std()


                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    df = df[df['PitchCall'] == 'InPlay']
    handVect.append(df['BatterSide'].iloc[0])
    metricName.append('90% EV')
    maxEV = df.groupby('BatterId').quantile(0.9)['ExitSpeed'].mean()
    metricValue.append(maxEV)
    if df['HomeTeam'].iloc[0] == "CLE_TIG":
        opponent = df['AwayTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    else:
        opponent = df['HomeTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    dateType.append('Season')
    dateValue.append('2023')
    hitter = df['Batter']
    hitterVect.append(hitter.iloc[0])
    dashboardVect.append('Quality Contact')
    team = df['BatterTeam']
    teamVect.append(team.iloc[0])
    if (maxEV >= avg_90 + std_90 * 1.5):
        compareToAverage.append(3)
    elif (maxEV <= avg_90 - std_90 * 1.5):
        compareToAverage.append(0)
    elif (maxEV <= avg_90 - std_90):
        compareToAverage.append(1)
    else:
        compareToAverage.append(2)

In [23]:
# hard hit%
# Find max velo across dataset for each hitter
maxVelo_dict = data.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].to_dict()

avg_hardHit = top_avg_HH.mean()
std_hardHit = top_avg_HH.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    # Finding max velo for entire season
    maxVelo = maxVelo_dict[bId]
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85    
    df = data[data['BatterId'] == bId]
    df = df[df['PitchCall'] == 'InPlay']
    handVect.append(df['BatterSide'].iloc[0])
    threshHits = df[df['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
    totalHits = df[df['ExitSpeed'].notna()].shape[0] # counts no. of hits
    metricName.append('HardHit%')          
    if totalHits == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        hardHitPercent = (threshHits * 100 / totalHits)
        metricValue.append(hardHitPercent) # hard hit percentage
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (hardHitPercent >= avg_hardHit + std_hardHit * 1.5):
            compareToAverage.append(3)
        elif (hardHitPercent <= avg_hardHit - std_hardHit * 1.5):
            compareToAverage.append(0)
        elif (hardHitPercent <= avg_hardHit - std_hardHit):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [24]:
avg_fbCon = top_avg_FBCon.mean()
std_fbCon = top_avg_FBCon.std()


                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    handVect.append(df['BatterSide'].iloc[0])
    FBs = df[df['TaggedPitchType'] == 'Fastball'].shape[0] # total fastballs
    fbContact = df[(df['TaggedPitchType'] == 'Fastball') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    metricName.append('FB Contact%')           
    if FBs == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        fbCon = (fbContact / FBs) * 100
        metricValue.append(fbCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (fbCon >= avg_fbCon + std_fbCon * 1.5):
            compareToAverage.append(3)
        elif (fbCon <= avg_fbCon - std_fbCon * 1.5):
            compareToAverage.append(0)
        elif (fbCon <= avg_fbCon - std_fbCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [25]:
avg_heartCon = top_avg_HeartCon.mean()
std_heartCon = top_avg_HeartCon.std()
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    handVect.append(df['BatterSide'].iloc[0])
    heartContact = df[(df['ZoneType'] == 'Heart') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    heartPitches = df[(df['ZoneType'] == 'Heart')].shape[0]
    metricName.append('Heart Contact%')        
    if heartPitches == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        heartCon = (heartContact / heartPitches) * 100
        metricValue.append(heartCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (heartCon >= avg_heartCon + std_heartCon * 1.5):
            compareToAverage.append(3)
        elif (heartCon <= avg_heartCon - std_heartCon * 1.5):
            compareToAverage.append(0)
        elif (heartCon <= avg_heartCon - std_heartCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

avg_IZCon = top_avg_IZCon.mean()
std_IZCon = top_avg_IZCon.std()

# in-zone contact %
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]    
    handVect.append(df['BatterSide'].iloc[0])
    zoneContact = ((df[(df['PitchCall'].isin(outcomes['Contact'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]))
    inZone = df[(df['ZoneType'].isin(outcomes['IZ']))].shape[0]
    metricName.append('InZone Contact%')       
    if inZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        IZCon = zoneContact * 100 / inZone
        metricValue.append(IZCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (IZCon >= avg_IZCon + std_IZCon * 1.5):
            compareToAverage.append(3)
        elif (IZCon <= avg_IZCon - std_IZCon * 1.5):
            compareToAverage.append(0)
        elif (IZCon <= avg_IZCon - std_IZCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [26]:
avg_MR = top_avg_MR.mean()
std_MR = top_avg_MR.std()

# MR %
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    df = df[df['PitchCall'] == 'InPlay']
    handVect.append(df['BatterSide'].iloc[0])
    mr = df[(df['Angle'] >= 10) & (df['Angle'] <= 30) & (df['ExitSpeed'] > 95)].shape[0]
    contact = df[df['PitchCall'].isin(outcomes['Contact'])].shape[0]
    metricName.append('MR%')    
    if contact == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        mr_percent = (mr / contact) * 100
        metricValue.append(mr_percent)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (mr_percent >= avg_MR + std_MR * 1.5):
            compareToAverage.append(3)
        elif (mr_percent <= avg_MR - std_MR * 1.5):
            compareToAverage.append(0)
        elif (mr_percent <= avg_MR - std_MR):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [27]:
avg_nonFBCon = top_avg_NonFBCon.mean()
std_nonFBCon = top_avg_NonFBCon.std()

# non-fb contact %
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    handVect.append(df['BatterSide'].iloc[0])
    nonFBs = df[df['TaggedPitchType'] != 'Fastball'].shape[0] # total NON fastballs
    nonFBContact = df[(df['TaggedPitchType'] != 'Fastball') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    metricName.append('Non-FB Contact%')
    if nonFBs == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        nonFBConPer = (nonFBContact / nonFBs) * 100
        metricValue.append(nonFBConPer)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (nonFBConPer >= avg_nonFBCon + std_nonFBCon * 1.5):
            compareToAverage.append(3)
        elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon * 1.5):
            compareToAverage.append(0)
        elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [28]:
avg_con = top_avg_contact.mean()
std_con = top_avg_contact.std()


# All Contact%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    contact = df[df['PitchCall'].isin(outcomes['Contact'])].shape[0]
    swings = df[df['PitchCall'].isin(outcomes['Swing'])].shape[0]
    metricName.append('Contact%')
    if swings == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        con_per = (contact / swings) * 100
        metricValue.append(con_per)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (con_per >= avg_con + std_con * 1.5):
            compareToAverage.append(3)
        elif (con_per <= avg_con - std_con * 1.5):
            compareToAverage.append(0)
        elif (con_per <= avg_con - std_con):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [29]:
avg_gb = top_avg_gb.mean()
std_gb = top_avg_gb.std()


# GB%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    inPlay = df[df['PitchCall'] == 'InPlay'].shape[0]
    inPlayDf = df[df['PitchCall'] == 'InPlay']
    gbs = inPlayDf[inPlayDf['Angle'] < 0].shape[0]
    metricName.append('GB%')
    if inPlay == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        gb_per = (gbs / inPlay) * 100
        metricValue.append(gb_per)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (gb_per >= avg_gb + std_gb * 1.5):
            compareToAverage.append(0)
        elif (gb_per <= avg_gb - std_gb * 1.5):
            compareToAverage.append(3)
        elif (gb_per <= avg_gb - std_gb):
            compareToAverage.append(2)
        else:
            compareToAverage.append(1)

In [30]:
#Validation Metric: Whiff%
##Do not include in dashboard

# Whiff%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    swings = df[df['PitchCall'].isin(outcomes['Swing'])].shape[0]
    swingsDf = df[df['PitchCall'].isin(outcomes['Swing'])]
    misses = swingsDf[swingsDf['PitchCall'] == 'StrikeSwinging'].shape[0]
    metricName.append('Whiff%')
    if swings == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        whiff_per = (misses / swings) * 100
        metricValue.append(whiff_per)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (gb_per >= avg_gb + std_gb * 1.5):
            compareToAverage.append(0)
        elif (gb_per <= avg_gb - std_gb * 1.5):
            compareToAverage.append(3)
        elif (gb_per <= avg_gb - std_gb):
            compareToAverage.append(2)
        else:
            compareToAverage.append(1)

In [31]:
#Validation Metric: Strike%
##Do not include in dashboard

# Strike%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    strikesSwing = df[df['PitchCall'] == 'StrikeSwinging'].shape[0]
    strikesTake = df[df['PitchCall'] == 'StrikeCalled'].shape[0]
    fouls = df[df['PitchCall'] == 'FoulBall'].shape[0]
    metricName.append('Strike%')
    strike_per = ((strikesSwing + strikesTake + fouls) / df.shape[0]) * 100
    metricValue.append(strike_per)
    if df['HomeTeam'].iloc[0] == "CLE_TIG":
        opponent = df['AwayTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    else:
        opponent = df['HomeTeam'].iloc[0]
        date = df['GameID'].iloc[0][4:8]
        gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
    dateType.append('Season')
    dateValue.append('2023')
    hitter = df['Batter']
    hitterVect.append(hitter.iloc[0])
    dashboardVect.append('Consistent Contact')
    team = df['BatterTeam']
    teamVect.append(team.iloc[0])
    if (gb_per >= avg_gb + std_gb * 1.5):
        compareToAverage.append(0)
    elif (gb_per <= avg_gb - std_gb * 1.5):
        compareToAverage.append(3)
    elif (gb_per <= avg_gb - std_gb):
        compareToAverage.append(2)
    else:
        compareToAverage.append(1)

In [32]:
""" Game Score
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    handVect.append(df['BatterSide'].iloc[0])
    metricName.append('Game Score')
    if swings == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        avgEVWeight = ((df['ExitSpeed'].mean() - avg_avgEV) / std_avgEV)*.1363154
        contactWeight = (((contact / swings) - avg_con) / std_con)*.16467817
        correctWeight = ((((correctSwings + correctTakes) / df.shape[0]) - avg_corrPer) / std_corrPer)*.13753384
        fbConWeight = (((nonFBContact / nonFBs) - avg_fbCon) / std_fbCon)*-.21622808
        maxEVWeight = ((df.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].mean() - avg_maxEV) / std_maxEV)*.02030876
        mrWeight = (((mr / contact) - avg_MR) / std_MR)*.18239217
        nonFBWeight = (((nonFBContact / nonFBs) - avg_nonFBCon) / std_nonFBCon)*-.08483979
        game_score = (100 + avgEVWeight + contactWeight + correctWeight + fbConWeight + maxEVWeight + mrWeight + nonFBWeight)
        metricValue.append(game_score)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Game Score')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (con_per >= avg_con + std_con * 1.5):
            compareToAverage.append(2)
        elif (con_per <= avg_con - std_con * 1.5):
            compareToAverage.append(0)
        else:
            compareToAverage.append(1)
"""

' Game Score\nfor bId in maxVelo_dict.keys():\n    hitterIDVect.append(bId)\n    df = data[data[\'BatterId\'] == bId] \n    handVect.append(df[\'BatterSide\'].iloc[0])\n    metricName.append(\'Game Score\')\n    if swings == 0:\n        metricValue.append(np.nan)\n        hitterVect.append(np.nan)\n        gameVect.append(np.nan)\n        dashboardVect.append(np.nan)\n        compareToAverage.append(np.nan)\n        dateType.append(np.nan)\n        dateValue.append(np.nan)\n        teamVect.append(np.nan)\n    else:\n        avgEVWeight = ((df[\'ExitSpeed\'].mean() - avg_avgEV) / std_avgEV)*.1363154\n        contactWeight = (((contact / swings) - avg_con) / std_con)*.16467817\n        correctWeight = ((((correctSwings + correctTakes) / df.shape[0]) - avg_corrPer) / std_corrPer)*.13753384\n        fbConWeight = (((nonFBContact / nonFBs) - avg_fbCon) / std_fbCon)*-.21622808\n        maxEVWeight = ((df.groupby(\'BatterId\').max(\'ExitSpeed\')[\'ExitSpeed\'].mean() - avg_maxEV) / std_maxEV

In [33]:
# Find wOBA
po = PitchOutcomes(data)
pitch_outcomes = po.compute_all()

pitch_outcomes_season = pd.DataFrame(pitch_outcomes.groupby('BatterId').sum()).reset_index()

#2023 wOBA Equation based on FanGraphs
numerator = (0.702*pitch_outcomes_season['Walks'] + 0.733*pitch_outcomes_season['HitByPitch'] + 0.892*pitch_outcomes_season['Singles'] 
         + 1.261*pitch_outcomes_season['Doubles'] + 1.593*pitch_outcomes_season['Triples'] + 2.039*pitch_outcomes_season['HomeRuns'])
denominator = pitch_outcomes_season['ABs']
pitch_outcomes_season['wOBA'] = numerator / denominator

pitch_outcomes_season

C:\Users\harri\Work Files\pitch_outcomes_batter.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[new_col_name] = self.data[mapping_col].map(mapping_dict)
C:\Users\harri\Work Files\pitch_outcomes_batter.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[new_col_name] = self.data[mapping_col].map(mapping_dict)
C:\Users\harri\Work Files\pitch_outcomes_batter.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,BatterId,BasesRecorded,ABs,TotalKs,TotalStrikes,TotalWhiffs,TotalSwings,Walks,HitByPitch,Singles,Doubles,Triples,HomeRuns,TotalSacrifices,wOBA
0,6.839530e+05,69.0,150,25.0,342.0,26.0,218.0,31.0,3.0,26.0,11.0,3.0,3.0,69.0,0.479467
1,6.864500e+05,52.0,107,14.0,249.0,44.0,200.0,5.0,1.0,16.0,6.0,0.0,6.0,52.0,0.358084
2,6.866110e+05,98.0,168,17.0,362.0,48.0,243.0,39.0,10.0,39.0,10.0,1.0,9.0,98.0,0.607440
3,6.873150e+05,40.0,89,12.0,176.0,34.0,140.0,12.0,4.0,12.0,4.0,0.0,5.0,40.0,0.419090
4,6.879180e+05,82.0,159,32.0,370.0,65.0,261.0,17.0,8.0,25.0,7.0,1.0,10.0,82.0,0.445962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,1.000236e+09,13.0,75,16.0,204.0,37.0,117.0,13.0,1.0,5.0,4.0,0.0,0.0,13.0,0.258173
970,1.000237e+09,53.0,108,8.0,236.0,20.0,170.0,6.0,1.0,29.0,8.0,0.0,2.0,53.0,0.416472
971,1.000238e+09,18.0,77,15.0,186.0,18.0,104.0,15.0,2.0,8.0,1.0,0.0,2.0,18.0,0.317805
972,1.000239e+09,19.0,76,12.0,173.0,25.0,107.0,14.0,2.0,5.0,1.0,0.0,3.0,19.0,0.304368


In [34]:
# Preparing Columns for Export

# Create DataFrame to merge back to offensive metrics
wobaTable = pd.DataFrame({'HitterID':pitch_outcomes_season['BatterId'], 'wOBAVal':pitch_outcomes_season['wOBA'], 'PA':pitch_outcomes_season['ABs']})

#re24Table = pd.DataFrame({'HitterID':re24['BatterId'], 'RE24':re24['RE24']})

# Offensive Metrics
metricsTable = pd.DataFrame({'Hitter':hitterVect, 'HitterID':hitterIDVect,'Game':gameVect, 'Metric':metricName, 'Dashboard': dashboardVect,
                         'Value':metricValue, 'CompareToAverage':compareToAverage, 'DateType':dateType, 'DateValue':dateValue, 
                         'Team':teamVect, 'Hand':handVect})


# Merge the Two
gameAvgs = pd.merge(metricsTable, wobaTable, how = 'inner', on = 'HitterID')

#Remove all NA Rows
gameAvgs = gameAvgs.dropna()
gameAvgs = gameAvgs[gameAvgs.DateValue != 'Error']

#Remove Duplicate rows (only use if a single metric for a player is needed, such as wOBA
#gameAvgs = gameAvgs.drop_duplicates(subset=['Hitter', 'HitterID'], keep='first')

#Export File
gameAvgs.to_excel('../Work Files/Clemson Offensive KPIs Across NCAA 2023.xlsx', index = False)
#gameAvgs.to_excel('../Work Files/NCAA wOBA 2023.xlsx', index = False)

In [35]:
data['ZoneType'].value_counts()

Chase    137683
Waste    131349
Edge     122474
Heart     42383
Name: ZoneType, dtype: int64

In [36]:
len(metricValue)

16558

In [37]:
#data.to_csv('../Work Files/NCAA Trackman 2023 (4-27).csv', index = False)